# 1. Setup

In [30]:
import os
import shutil
import pandas as pd
import kagglehub

In [31]:
# Download latest version
path = kagglehub.dataset_download("yasserh/loan-default-dataset")
print("Path to dataset files:", path)

Path to dataset files: /Users/giliamo/.cache/kagglehub/datasets/yasserh/loan-default-dataset/versions/1


In [32]:
# Download latest version
path = kagglehub.dataset_download("yasserh/loan-default-dataset")
print("Path to dataset files:", path)

# Set the target directory for storing the specific CSV file
target_dir = "/Users/giliamo/code/YannAll/raw_data"
os.makedirs(target_dir, exist_ok=True)

# Specify the name of the CSV file you want to move
csv_file_name = "Loan_Default.csv"

# Move the CSV file to the target directory (assumes the file exists)
shutil.move(os.path.join(path, csv_file_name), os.path.join(target_dir, csv_file_name))

# Load the CSV file
data = pd.read_csv(os.path.join(target_dir, csv_file_name))

Path to dataset files: /Users/giliamo/.cache/kagglehub/datasets/yasserh/loan-default-dataset/versions/1


FileNotFoundError: [Errno 2] No such file or directory: '/Users/giliamo/.cache/kagglehub/datasets/yasserh/loan-default-dataset/versions/1/Loan_Default.csv'

In [24]:
data.head()

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
0,24890,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,...,EXP,758,CIB,25-34,to_inst,98.728814,south,direct,1,45.0
1,24891,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,...,EQUI,552,EXP,55-64,to_inst,NaN,North,direct,1,NaN
2,24892,2019,cf,Male,pre,type1,p1,l1,nopc,nob/c,...,EXP,834,CIB,35-44,to_inst,80.019685,south,direct,0,46.0
3,24893,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,...,EXP,587,CIB,45-54,not_inst,69.376900,North,direct,0,42.0
4,24894,2019,cf,Joint,pre,type1,p1,l1,nopc,nob/c,...,CRIF,602,EXP,25-34,not_inst,91.886544,North,direct,0,39.0


# 2. EDA

### 2.1 Data Check

In [33]:
data.shape

(148670, 34)

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148670 entries, 0 to 148669
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ID                         148670 non-null  int64  
 1   year                       148670 non-null  int64  
 2   loan_limit                 145326 non-null  object 
 3   Gender                     148670 non-null  object 
 4   approv_in_adv              147762 non-null  object 
 5   loan_type                  148670 non-null  object 
 6   loan_purpose               148536 non-null  object 
 7   Credit_Worthiness          148670 non-null  object 
 8   open_credit                148670 non-null  object 
 9   business_or_commercial     148670 non-null  object 
 10  loan_amount                148670 non-null  int64  
 11  rate_of_interest           112231 non-null  float64
 12  Interest_rate_spread       112031 non-null  float64
 13  Upfront_charges            10

### 2.2 Missing Values

In [34]:
missing_values = data.isnull().sum()
missing_percentage = (missing_values / len(data)) * 100

# Combine both counts and percentages into a DataFrame for better readability
missing_summary = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage (%)': missing_percentage
})

# Filter to show only columns with at least 1 missing value and sort by percentage in descending order
missing_summary = missing_summary[missing_summary['Missing Values'] > 0].sort_values(by='Percentage (%)', ascending=False)

print(missing_summary)

                           Missing Values  Percentage (%)
Upfront_charges                     39642       26.664425
Interest_rate_spread                36639       24.644515
rate_of_interest                    36439       24.509989
dtir1                               24121       16.224524
property_value                      15098       10.155378
LTV                                 15098       10.155378
income                               9150        6.154571
loan_limit                           3344        2.249277
approv_in_adv                         908        0.610749
submission_of_application             200        0.134526
age                                   200        0.134526
loan_purpose                          134        0.090133
Neg_ammortization                     121        0.081388
term                                   41        0.027578
